# Hugging Face tokenizers

La librería `tokenizers` de Hugging Face proporciona una implementación de los tokenizadores más utilizados en la actualidad, centrándose en el rendimiento y la versatilidad.

## Instalación

Para instalar `tokenizers` con pip:

```bash
pip install tokenizers
```

para instalar `tokenizers` con conda:

```bash
conda install conda-forge::tokenizers
```

## El pipeline de tokenización

Para tokenizar una secuencia se usa `Tokenizer.encode`, el cual realiza los siguientes pasos:

 * Normalización
 * pre-tokenización
 * Tokenización
 * Post-tokenización

Vamos a ver cada una

Para realizar el post vamos a usar el dataset [wikitext-103](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/)

In [2]:
!wget https://dax-cdn.cdn.appdomain.cloud/dax-wikitext-103/1.0.1/wikitext-103.tar.gz

--2024-02-26 08:14:11--  https://dax-cdn.cdn.appdomain.cloud/dax-wikitext-103/1.0.1/wikitext-103.tar.gz
Resolving dax-cdn.cdn.appdomain.cloud (dax-cdn.cdn.appdomain.cloud)... 23.200.169.125
Connecting to dax-cdn.cdn.appdomain.cloud (dax-cdn.cdn.appdomain.cloud)|23.200.169.125|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 189603606 (181M) [application/x-gzip]
Saving to: ‘wikitext-103.tar.gz’

wikitext-103.tar.gz 100%[===================>] 180,82M  6,42MB/s    in 30s     

2024-02-26 08:14:42 (5,95 MB/s) - ‘wikitext-103.tar.gz’ saved [189603606/189603606]



In [3]:
!tar -xvzf wikitext-103.tar.gz

wikitext-103/
wikitext-103/wiki.test.tokens
wikitext-103/wiki.valid.tokens
wikitext-103/README.txt
wikitext-103/LICENSE.txt
wikitext-103/wiki.train.tokens


In [4]:
!rm wikitext-103.tar.gz

### Normalización

La normalización son operaciones que se aplican al texto antes de la tokenización, como la eliminación de espacios en blanco, la conversión a minúsculas, la eliminación de caracteres especiales, etc. En Hugging Face están implementadas las siguientes normalizaciones:

|Normalización|Descripción|Ejemplo|
|---|---|---|
|NFD (Normalization for D)|Los caracteres se descomponen por equivalencia canónica|`â` (U+00E2) se descompone en `a` (U+0061) + `^` (U+0302)|
|NFKD (Normalization Form KD)|Los caracteres se descomponen por compatibilidad|`ﬁ` (U+FB01) se descompone en `f` (U+0066) + `i` (U+0069)|
|NFC (Normalization Form C)|Los caracteres se descomponen y luego se recomponen por equivalencia canónica|`â` (U+00E2) se descompone en `a` (U+0061) + `^` (U+0302) y luego se recompone en `â` (U+00E2)|
|NFKC (Normalization Form KC)|Los caracteres se descomponen por compatibilidad y luego se recomponen por equivalencia canónica|`ﬁ` (U+FB01) se descompone en `f` (U+0066) + `i` (U+0069) y luego se recompone en `f` (U+0066) + `i` (U+0069)|
|Lowercase|Convierte el texto a minúsculas|`Hello World` se convierte en `hello world`|
|Strip|Elimina todos los espacios en blanco de los lados especificados (izquierdo, derecho o ambos) del texto|`  Hello World  ` se convierte en `Hello World`|
|StripAccents|Elimina todos los símbolos de acento en unicode (se utilizará con NFD por coherencia)|`á` (U+00E1) se convierte en `a` (U+0061)|
|Replace|Sustituye una cadena personalizada o [regex](https://maximofn.com/regular-expressions/) y la cambia por el contenido dado|`Hello World` se convierte en `Hello Universe`|
|BertNormalizer|Proporciona una implementación del Normalizador utilizado en el BERT original. Las opciones que se pueden configurar son `clean_text`, `handle_chinese_chars`, `strip_accents` y `lowercase`|`Hello World` se convierte en `hello world`|

Vamos a crear un normalizador para ver cómo funciona

In [6]:
from tokenizers import normalizers

bert_normalizer = normalizers.BertNormalizer()

input_text = "Héllò hôw are ü?"
normalized_text = bert_normalizer.normalize_str(input_text)
normalized_text

'hello how are u?'

Para usar varios normalizadores podemos usar el método `Sequence`

In [7]:
custom_normalizer = normalizers.Sequence([normalizers.NFKC(), normalizers.BertNormalizer()])

normalized_text = custom_normalizer.normalize_str(input_text)
normalized_text

'hello how are u?'

Para modificar el normalizador de un tokenizador

In [9]:
import tokenizers

tokenizer = tokenizers.BertWordPieceTokenizer()

In [10]:
tokenizer.normalizer = custom_normalizer

### Pre-tokenización

La pretokenización es el acto de dividir un texto en objetos más pequeños. El pretokenizador dividirá el texto en "palabras" y los tokens finales serán partes de esas palabras.

El PreTokenizer se encarga de dividir la entrada según un conjunto de reglas. Este preprocesamiento le permite asegurarse de que el tokenizador no construye tokens a través de múltiples "divisiones". Por ejemplo, si no quieres tener espacios en blanco dentro de un token, entonces puedes tener un rre tokenizer que divide en las palabras a partir de espacios en blanco.

En Hugging Face están implementados los siguientes pre tokenizadores

|PreTokenizer|Descripción|Ejemplo|
|---|---|---|
|ByteLevel|Divide en espacios en blanco mientras reasigna todos los bytes a un conjunto de caracteres visibles. Esta técnica fue introducida por OpenAI con GPT-2 y tiene algunas propiedades más o menos buenas: Como mapea sobre bytes, un tokenizador que utilice esto sólo requiere 256 caracteres como alfabeto inicial (el número de valores que puede tener un byte), frente a los más de 130.000 caracteres Unicode. Una consecuencia del punto anterior es que es absolutamente innecesario tener un token desconocido usando esto ya que podemos representar cualquier cosa con 256 tokens. Para caracteres no ascii, se vuelve completamente ilegible, ¡pero funciona!|`Hello my friend, how are you?` se divide en `Hello`, `Ġmy`, Ġfriend`, `,`, `Ġhow`, `Ġare`, `Ġyou`, `?`|
|Whitespace|Divide en límites de palabra usando la siguiente expresión regular: `\w+|[^\w\s]+`. En mi post sobre [expresiones regulares](https://maximofn.com/regular-expressions/) puedes entender qué hace|`Hello there!` se divide en `Hello`, `there`, `!`|
|WhitespaceSplit|Se divide en cualquier carácter de espacio en blanco|`Hello there!` se divide en `Hello`, `there!`|
|Punctuation|Aislará todos los caracteres de puntuación|`Hello?` se divide en `Hello`, `?`|
|Metaspace|Separa los espacios en blanco y los sustituye por un carácter especial "▁" (U+2581)|`Hello there` se divide en `Hello`, `▁there`|
|CharDelimiterSplit|Divisiones en un carácter determinado|Ejemplo con el caracter `x`: `Helloxthere` se divide en `Hello`, `there`|
|Digits|Divide los números de cualquier otro carácter|`Hello123there` se divide en `Hello`, `123`, `there`|
|Split|Pretokenizador versátil que divide según el patrón y el comportamiento proporcionados. El patrón se puede invertir si es necesario. El patrón debe ser una cadena personalizada o una [regex](https://maximofn.com/regular-expressions/). El comportamiento debe ser `removed`, `isolated`, `merged_with_previous`, `merged_with_next`, `contiguous`. Para invertir se indica con un booleano|Ejemplo con pattern=`" "`, behavior=`isolated`, invert=`False`: `Hello, how are you?` se divide en `Hello,`, ` `, `how`, ` `, `are`, ` `, `you?`|

Vamos a crear un pre tokenizador para ver cómo funciona

In [8]:
from tokenizers import pre_tokenizers

pre_tokenizer = pre_tokenizers.Digits(individual_digits=True)

input_text = "I paid $30 for the car"
pre_tokenized_text = pre_tokenizer.pre_tokenize_str(input_text)
pre_tokenized_text

[('I paid $', (0, 8)),
 ('3', (8, 9)),
 ('0', (9, 10)),
 (' for the car', (10, 22))]

Para usar varios pre tokenizadores podemos usar el método `Sequence`

In [12]:
custom_pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Whitespace(), pre_tokenizers.Digits(individual_digits=True)])

pre_tokenized_text = custom_pre_tokenizer.pre_tokenize_str(input_text)
pre_tokenized_text

[('I', (0, 1)),
 ('paid', (2, 6)),
 ('$', (7, 8)),
 ('3', (8, 9)),
 ('0', (9, 10)),
 ('for', (11, 14)),
 ('the', (15, 18)),
 ('car', (19, 22))]

Para modificar el pre tokenizador de un tokenizador

In [13]:
tokenizer.pre_tokenizer = custom_pre_tokenizer

### Tokenización